In [1]:
import wandb
import pandas as pd
import numpy as np
import pathlib

In [10]:
!wget 'https://www.statssa.gov.za/timeseriesdata/Excel/P0141%20-%20CPI(5%20and%208%20digit)%20from%20Jan%202017%20(202307).zip'

--2023-08-28 08:14:23--  https://www.statssa.gov.za/timeseriesdata/Excel/P0141%20-%20CPI(5%20and%208%20digit)%20from%20Jan%202017%20(202307).zip
Resolving www.statssa.gov.za (www.statssa.gov.za)... 45.60.125.5, 45.60.123.5
Connecting to www.statssa.gov.za (www.statssa.gov.za)|45.60.125.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 212 [text/html]
Saving to: ‘P0141 - CPI(5 and 8 digit) from Jan 2017 (202307).zip’

P0141 - CPI(5 and 8 100%[===================>]     212  --.-KB/s    in 0s      

2023-08-28 08:14:23 (67.4 MB/s) - ‘P0141 - CPI(5 and 8 digit) from Jan 2017 (202307).zip’ saved [212/212]



In [12]:
!unzip 'P0141 - CPI(5 and 8 digit) from Jan 2017 (202307).zip'

Archive:  P0141 - CPI(5 and 8 digit) from Jan 2017 (202307).zip
  inflating: EXCEL - CPI(5 and 8 digit) from Jan 2017 (202307).xlsx  


In [13]:
run = wandb.init(
    project="zindi-rmb", 
    job_type='upload-dataset'
    )

In [14]:
artifact = wandb.Artifact(
    name='cpi.csv', 
    type='dataset'
    )  

In [15]:
# get directory path
path = str(pathlib.Path().cwd().parent.parent.resolve())

In [16]:
# load raw data from stastsa
raw_df = pd.read_excel(
    "EXCEL - CPI(5 and 8 digit) from Jan 2017 (202307).xlsx",
    dtype="object",
)

There are 12 main categories that are used to calculate the Headline CPI. Each category has a number of sub categories. This data set contains the CPI on sub-category level.

Create a dataset where the CPI is calculated on the category level. These values will then be used in the models to predict the CPI for the different categories as well as the Headline CPI.

In terms of cleaning the data, I will do the following:

Remove unnecessary columns.
Change column headers to make them more explanatory.
Replace all .. entries with a 0. There are products which were included in the CPI calculations at a later stage and some products that were removed from the CPI calculation. Categories with no value at the time has a .., I will replace them with a 0. The type of the column can then be updated to float.
Combine the Super maize and Special maize categories into a single Maize meal category, to correspond with the current use of maize meal.
Create a function to calculate the CPI values for each month using the weights provided in the file

In [17]:
def get_montly_cpi(raw_df):
    """Function that takes the raw cpi data for each product from statssa and calculates the cpi value per category

    Arguments:
    ----------
    raw_df: pandas dataframe
            dataframe containing raw data from statsa

    Return:
    -------
    df_cpi: pandas dataframe
            dataframe containing the monthly cpi per category
    """

    # 1. remove unecessary columns and rename
    list_cols_to_drop = ["H01", "H02", "H05", "H06", "H07"]
    cat_cpi_df = raw_df.copy().drop(list_cols_to_drop, axis=1).copy()

    cat_cpi_df.rename(
        columns={
            "H03": "category_codes",
            "H04": "category_descr",
            "Weight (All urban)": "weights_urban",
        },
        inplace=True,
    )

    # 2. replace .. with zeros
    cat_cpi_df.replace("..", 0, inplace=True)

    # 3. combine maize meal categories
    cat_cpi_df.iloc[17:19] = (
        cat_cpi_df.iloc[17:19].copy().apply(pd.to_numeric, errors="coerce")
    )
    divided_row = (cat_cpi_df.iloc[17].copy() + cat_cpi_df.iloc[18].copy()) / 2
    cat_cpi_df.iloc[15] = [
        divided_row[i] if value == 0 else value
        for i, value in enumerate(cat_cpi_df.iloc[15].copy())
    ]
    cat_cpi_df.drop([cat_cpi_df.index[17], cat_cpi_df.index[18]], inplace=True)

    # Convert the 'weights_urban' column to float
    cat_cpi_df["weights_urban"] = cat_cpi_df["weights_urban"].astype("float")

    # 4. calculate cpi
    # Assign a main category code to each raw data row.
    main_category = []
    for index, row in cat_cpi_df.iterrows():
        if (len(row["category_codes"]) == 8) & (
            row["category_codes"][:2] in ["01", "02"]
        ):
            main_category.append(row["category_codes"][:2])
        elif (
            len(row["category_codes"]) == 5
        ):  # & (row['category_codes'][:2] not in ["04","07"]):
            main_category.append(row["category_codes"][:2])
        else:
            main_category.append("no")

    cat_cpi_df["main_category_code"] = main_category

    # Drop the rows where the main_category_code is "no". That is to prevent double counting.
    # Some categories have a sub category included in the data.
    cat_cpi_df.drop(
        cat_cpi_df[cat_cpi_df["main_category_code"] == "no"].index, inplace=True
    )

    # Sum the weights for each category
    sum_weights = cat_cpi_df.groupby("main_category_code")["weights_urban"].sum()

    # create new cpi dataframe
    cpi_df = pd.DataFrame()

    # For each month create the headline CPI value and the CPI value per category.
    for col in range(3, cat_cpi_df.shape[1] - 1):
        cat_cpi_df = cat_cpi_df.copy()
        column_name = cat_cpi_df.columns[col]
        cat_cpi_df["weighted_index_" + column_name] = (
            cat_cpi_df["weights_urban"] * cat_cpi_df[column_name]
        )

        sum_weighted_index = cat_cpi_df.groupby("main_category_code")[
            "weighted_index_" + column_name
        ].sum()

        # Concatenate the DataFrames horizontally
        concat_df = pd.concat([sum_weights, sum_weighted_index], axis=1)

        # Add a row that sums the values in the columns
        sums_df = pd.DataFrame(
            concat_df.sum().values.reshape(1, -1), columns=concat_df.columns
        )
        sums_df = sums_df.set_index(pd.Index(["headline"]))

        # Concatenate the headline dataframe to the categories
        month_cpi_df = pd.concat([concat_df, sums_df], axis=0)

        # Calculate the CPI value
        month_cpi_df["cpi_" + column_name] = (
            month_cpi_df["weighted_index_" + column_name]
            / month_cpi_df["weights_urban"]
        ).round(1)

        cpi_df = pd.concat(
            [cpi_df, month_cpi_df[["weights_urban", "cpi_" + column_name]]], axis=1
        )

    # Remove duplicate weights columns and reset the index
    cpi_df = cpi_df.loc[:, ~cpi_df.columns.duplicated()]
    cpi_df = cpi_df.reset_index().rename(columns={"index": "category"})

    # Dataframe with just the CPI values:
    cpi_df = cpi_df.drop("weights_urban", axis=1).copy()
    transposed_cpi_df = cpi_df.set_index("category").transpose().reset_index()
    transposed_cpi_df["date"] = transposed_cpi_df["index"].apply(
        lambda x: x.split("M")[-1]
    )
    transposed_cpi_df["date"] = transposed_cpi_df["date"].apply(
        lambda x: x[:4] + "-" + x[-2:]
    )
    # change month to datetime format
    transposed_cpi_df["date"] = pd.to_datetime(transposed_cpi_df["date"]).dt.strftime(
        "%Y-%m"
    )

    return transposed_cpi_df

In [18]:
cpi = get_montly_cpi(raw_df=raw_df)

In [19]:
# Create a category dictionary with the category code and description
category_dict = {
    "01": "Food and non-alcoholic beverages",
    "02": "Alcoholic beverages and tobacco",
    "03": "Clothing and footwear",
    "04": "Housing and utilities",
    "05": "Household contents and services",
    "06": "Health",
    "07": "Transport",
    "08": "Communication",
    "09": "Recreation and culture",
    "10": "Education",
    "11": "Restaurants and hotels",
    "12": "Miscellaneous goods and services",
    "headline": "headline CPI",
}

In [20]:
cpi = cpi.rename(columns=category_dict)

In [21]:
cpi.head()

category,index,Food and non-alcoholic beverages,Alcoholic beverages and tobacco,Clothing and footwear,Housing and utilities,Household contents and services,Health,Transport,Communication,Recreation and culture,Education,Restaurants and hotels,Miscellaneous goods and services,headline CPI,date
0,cpi_M201701,81.6,78.5,92.7,82.1,90.5,79.3,76.7,103.7,95.9,74.2,88.2,78.4,82.4,2017-01
1,cpi_M201702,82.2,78.3,93.0,82.1,90.2,82.3,77.3,103.6,96.1,74.2,89.3,81.6,83.1,2017-02
2,cpi_M201703,82.4,79.5,93.3,82.8,90.7,83.0,77.2,102.9,96.2,79.4,89.4,81.7,83.6,2017-03
3,cpi_M201704,82.3,80.1,93.3,82.8,90.5,83.4,77.0,102.9,96.4,79.4,89.2,82.0,83.6,2017-04
4,cpi_M201705,82.7,80.3,93.5,82.8,90.6,83.6,77.8,102.7,96.3,79.4,88.5,82.0,83.8,2017-05


In [22]:
cpi.to_csv("cpi.csv")

In [23]:
artifact.add_file(local_path='cpi.csv')
run.log_artifact(artifact)

In [24]:
run.finish()